# Wikidata にある日本の市区町村データの整合性チェック

Wikidata にある日本の市区町村データと、総務省サイトにあるそれとを比較し、一致することを確認する。

In [2]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, XML

# 表のすべての行を表示する
# pd.set_option('display.max_rows', None)

## 総務省サイトからすべての市区町村の一覧を取得

In [3]:
# 総務省のサイト https://www.soumu.go.jp/denshijiti/code.html から
# 最新の「都道府県コード及び市区町村コード」を読み込む。
excel_file_url = "https://www.soumu.go.jp/main_content/000894847.xls"

# １枚目のシート。市町村と東京特別区。普通の区は含まない。
df_municipalities_sub = pd.read_excel(excel_file_url, sheet_name=0)
# カラム名を変更
df_municipalities_sub.rename(columns={'都道府県名\n（漢字）': '都道府県名', '市区町村名\n（漢字）': '市区町村名'}, inplace=True)
# 市区町村名のカラムが　NaN の行は都道府県なので削除する
df_municipalities_sub.dropna(subset=['市区町村名'], inplace=True)

# ２枚目のシート。普通の区。
df_wards = pd.read_excel(excel_file_url, sheet_name=1)
# カラム名を変更
df_wards.rename(columns={'都道府県名\n（漢字）': '都道府県名', '市区町村名\n（漢字）': '市区町村名', '都道府県名\n（ｶﾅ）': '都道府県名\n（カナ）', '市区町村名\n（ｶﾅ）': '市区町村名\n（カナ）'}, inplace=True)
# 市区町村名のカラムが「市」で終わる行は市なので削除する
df_wards = df_wards.query('not 市区町村名.str.endswith("市")')

# 二つを合わせてすべての自治体のリストを得る
df_municipalities = pd.concat([df_municipalities_sub, df_wards],ignore_index=True)
df_municipalities.sort_values('団体コード', ascending=True, inplace=True)
df_municipalities.reset_index(inplace=True, drop=True)
df_municipalities

Wikidata から SPARQL 言語による検索結果を RDF(JSON) で得る関数を定義する。

In [6]:
def QuerySparql(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

## 市の数のチェック

In [30]:
# 都道府県ごとの市の数
num_cities_query = """
SELECT ?pref ?prefLabel ?prefCode (COUNT(DISTINCT ?city) AS ?count) WHERE {
  ?city wdt:P31/wdt:P279* wd:Q494721 . # ?city は「日本の市」またはその部分集合（の部分集合……）の要素である。
  MINUS { ?city wdt:P576 ?abolished. } # ただし「廃止日」がないこと。
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?city wdt:P131+ ?pref. # ?pref に位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  OPTIONAL { ?pref wdt:P429 ?prefCode. } # 都道府県の全国地方公共団体コード
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "ja" .
  }
}
GROUP BY ?pref ?prefLabel ?prefCode
ORDER BY ASC(?prefCode)
"""
json_num_cities_wikidata = QuerySparql(num_cities_query)

# pandas DataFrame へ変換
num_cities_wikidata = []
for c in json_num_cities_wikidata:
    city = {
        '都道府県名': c['prefLabel']['value'],
        '市の数': int(c['count']['value'])
    }
    num_cities_wikidata.append(city)
df_num_cities_wikidata = pd.DataFrame(num_cities_wikidata)
df_num_cities_wikidata

,都道府県名,市の数
0,北海道,35
1,青森県,10
2,岩手県,14
3,宮城県,14
4,秋田県,13
5,山形県,13
6,福島県,13
7,茨城県,32
8,栃木県,14
9,群馬県,12


これらの Wikidata による市の数を、総務省のページ
https://www.j-lis.go.jp/spd/code-address/kenbetsu-inspection/cms_11914151.html
にある数と比べ、一致することを確認すること。

In [35]:
# すべての自治体の一覧を Wikidata から取得
municipalities_query = """
SELECT DISTINCT ?pref ?prefLabel ?city ?cityLabel ?district ?districtLabel ?municipality ?municipalityLabel ?code WHERE {
  {
    # 以下のような例を得るために、wdt:P31 のあとに "/wdt:P279*" が必要。
    # 山形市 P31 中核市 P279 日本の市
    # 所沢市 P31 特例市 P279 日本の市
    ?municipality wdt:P31/wdt:P279* wd:Q494721 . # ?municipality は「日本の市」の（直接または間接的な）要素である。
  } 
  UNION
  {?municipality wdt:P31 wd:Q5327704 . } # ?municipality は「日本の特別区」の要素である。
  UNION
  {?municipality wdt:P31 wd:Q137773 . } # ?municipality は「日本の区」の要素である。
  UNION
  {?municipality wdt:P31 wd:Q1059478 . } # ?municipality は「日本の町」の要素である。
  UNION
  {?municipality wdt:P31 wd:Q4174776 . } # ?municipality は「日本の村」の要素である。
  MINUS { ?municipality wdt:P576 ?abolished . } # ただし「廃止日」がないこと。

  OPTIONAL { ?municipality wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    ?municipality wdt:P131 ?city. # ?municipality は ?city に位置する。
    ?city wdt:P31 wd:Q494721. # ?city は「日本の市」である。
  }
  OPTIONAL {
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?municipality wdt:P131+ ?pref. # ?pref に（直接または間接的に）位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  OPTIONAL {
    ?municipality p:P131 ?statement1.
    ?statement1 ps:P131 ?district. # ?municipality は ?district に位置する。
    ?district wdt:P31 wd:Q1122846. # ?district は「郡」である。
    MINUS { ?statement1 pq:P582 ?endTime. } # 「終了日」が修飾子にあるものは除く。
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_municipalities_wikidata = QuerySparql(municipalities_query)
json_municipalities_wikidata

[{'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'},
  'municipality': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q37951'},
  'municipalityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '札幌市'},
  'code': {'type': 'literal', 'value': '011002'}},
 {'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'},
  'municipality': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q1079635'},
  'municipalityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '中央区'},
  'code': {'type': 'literal', 'value': '011011'},
  'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37951'},
  'cityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '札幌市'}},
 {'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type'

In [36]:
# pandas DataFrame へ変換
muni_wikidata = []
for item in json_municipalities_wikidata:
    muni = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None),
        '市名': (item['cityLabel']['value'] if ('cityLabel' in item) else None),
        '郡名': (item['districtLabel']['value'] if ('districtLabel' in item) else None),
        '自治体名': item['municipalityLabel']['value']
    }
    muni_wikidata.append(muni)
df_municipalities_wikidata = pd.DataFrame(muni_wikidata)
df_municipalities_wikidata

,団体コード,都道府県名,市名,郡名,自治体名
0,11002,北海道,None,None,札幌市
1,11011,北海道,札幌市,None,中央区
2,11029,北海道,札幌市,None,北区
3,11037,北海道,札幌市,None,東区
4,11045,北海道,札幌市,None,白石区
...,...,...,...,...,...
1922,473618,沖縄県,None,島尻郡,久米島町
1923,473626,沖縄県,None,島尻郡,八重瀬町
1924,473758,沖縄県,None,宮古郡,多良間村
1925,473812,沖縄県,None,八重山郡,竹富町


In [37]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
#
# 2023/10/01 現在、浜松市の2024年に予定されている統合前の区7つがここに表示される。
df_municipalities_only_in_wikidata = df_municipalities_wikidata[~df_municipalities_wikidata['団体コード'].isin(df_municipalities['団体コード'])]
df_municipalities_only_in_wikidata

,団体コード,都道府県名,市名,郡名,自治体名
1030,221317,静岡県,浜松市,None,中区
1031,221325,静岡県,浜松市,None,東区
1032,221333,静岡県,浜松市,None,西区
1033,221341,静岡県,浜松市,None,南区
1034,221350,静岡県,浜松市,None,北区
1035,221368,静岡県,浜松市,None,浜北区
1036,221376,静岡県,浜松市,None,天竜区


In [38]:
# 総務省のデータにあって Wikidata にはないものを表示
# 何もないのが正しい。
#
# 2023/10/01 現在、浜松市の2024年に予定されている統合で団体コードがつけ変わる天竜区がここに表示される。
df_municipalities_not_in_wikidata = df_municipalities[~df_municipalities['団体コード'].isin(df_municipalities_wikidata['団体コード'])]
df_municipalities_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）,Unnamed: 5
1029,221406,静岡県,浜松市天竜区,ｼｽﾞｵｶｹﾝ,ﾊﾏﾏﾂｼﾃﾝﾘｭｳｸ,R6.1.1施行
